In [ ]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

<ipython-input-5-fbd3e3930b1e>:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [ ]:
# Path to your dataset
dataset_path = '/content/drive/MyDrive/ADS Project/update/processed_images'

# Prepare ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # assuming 20% of the data is used for validation
)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),  # Resize all images to this size
    batch_size=32,
    class_mode='categorical',  # Multi-class classification
    subset='training'  # Set as training data
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

Found 2228 images belonging to 5 classes.
Found 553 images belonging to 5 classes.


In [ ]:
class CNNHyperModel(HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential([
            Conv2D(
                filters=hp.Int('conv_1_filters', min_value=32, max_value=128, step=32),
                kernel_size=(3, 3),
                activation='relu',
                input_shape=(150, 150, 3)
            ),
            MaxPooling2D(2, 2),
            Conv2D(
                filters=hp.Int('conv_2_filters', min_value=64, max_value=256, step=32),
                kernel_size=(3, 3),
                activation='relu'
            ),
            MaxPooling2D(2, 2),
            Conv2D(
                filters=hp.Int('conv_3_filters', min_value=128, max_value=512, step=32),
                kernel_size=(3, 3),
                activation='relu'
            ),
            MaxPooling2D(2, 2),
            Flatten(),
            Dense(
                units=hp.Int('dense_units', min_value=512, max_value=1024, step=256),
                activation='relu'
            ),
            Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)),
            Dense(self.num_classes, activation='softmax')
        ])

        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            ),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

# Prepare ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

hypermodel = CNNHyperModel(num_classes=train_generator.num_classes)

# Random Search
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='hparam_tuning'
)

tuner.search(train_generator, epochs=10, validation_data=validation_generator)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

Found 2228 images belonging to 5 classes.
Found 553 images belonging to 5 classes.

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
128               |128               |conv_1_filters
160               |160               |conv_2_filters
352               |352               |conv_3_filters
512               |512               |dense_units
0.1               |0.1               |dropout
0.00086293        |0.00086293        |learning_rate



KeyboardInterrupt: 

In [ ]:
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
best_hyperparameters = best_trial.hyperparameters.values

print("Best Random Search Model Hyperparameters::", best_hyperparameters)
print("Best Validation Accuracy Using Random Search:", best_trial.score)

Best Random Search Model Hyperparameters:: {'conv_1_filters': 64, 'conv_2_filters': 256, 'conv_3_filters': 224, 'dense_units': 1024, 'dropout': 0.4, 'learning_rate': 0.0018077173864559889}
Best Validation Accuracy Using Random Search: 0.719710648059845


In [ ]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

In [ ]:
def build_model(hp):
    model = Sequential([
        Conv2D(
            filters=hp.Int('conv_1_filters', min_value=32, max_value=64, step=32),
            kernel_size=hp.Choice('conv_1_kernel_size', values=[3]),
            activation='relu',
            input_shape=(150, 150, 3)
        ),
        MaxPooling2D(2, 2),
        Conv2D(
            filters=hp.Int('conv_2_filters', min_value=64, max_value=128, step=32),
            kernel_size=hp.Choice('conv_2_kernel_size', values=[3]),
            activation='relu'
        ),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(
            units=hp.Int('dense_units', min_value=128, max_value=256, step=128),
            activation='relu'
        ),
        Dropout(hp.Float('dropout', min_value=0.1, max_value=0.2, step=0.1)),
        Dense(10, activation='softmax')  # Adjust according to your class count
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
from keras_tuner import BayesianOptimization

bayesian_tuner = BayesianOptimization(
    hypermodel,  # Your updated hypermodel function or class
    objective='val_accuracy',
    max_trials=10,
    directory='output',
    project_name='bayesian_optimization'
)

In [ ]:
from keras_tuner import Hyperband

hyperband_tuner = Hyperband(
    hypermodel,  # Your updated hypermodel function or class
    objective='val_accuracy',
    max_epochs=10,  # Adjust based on what your GPU can handle and your patience
    directory='output',
    project_name='hyperband'
)

In [ ]:
# Running Bayesian Optimization
bayesian_tuner.search(train_generator, validation_data=validation_generator)

In [ ]:
# Get the best hyperparameters from Bayesian Optimization
best_hyperparameters_bayesian = bayesian_tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Bayesian Search Model Hyperparameters:", best_hyperparameters_bayesian.values)
best_trial_bayesian = bayesian_tuner.oracle.get_best_trials(num_trials=1)[0]
print("Best Validation Accuracy Using Bayesian Search:", best_trial_bayesian.score)

Best Bayesian Search Model Hyperparameters: {'conv_1_filters': 32, 'conv_2_filters': 256, 'conv_3_filters': 384, 'dense_units': 768, 'dropout': 0.5, 'learning_rate': 0.0023502267630575342}
Best Validation Accuracy Using Bayesian Search: 0.6220614910125732


In [ ]:
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=16,  # Reduced batch size
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=16,  # Reduced batch size
    class_mode='categorical',
    subset='validation'
)

Found 2228 images belonging to 5 classes.
Found 553 images belonging to 5 classes.


In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

Physical devices cannot be modified after being initialized


In [ ]:
# Running Hyperband
hyperband_tuner.search(train_generator, validation_data=validation_generator)

Trial 10 Complete [00h 02m 55s]
val_accuracy: 0.43942132592201233

Best val_accuracy So Far: 0.70705246925354
Total elapsed time: 00h 44m 04s

Search: Running Trial #11

Value             |Best Value So Far |Hyperparameter
64                |64                |conv_1_filters
256               |256               |conv_2_filters
512               |416               |conv_3_filters
1024              |512               |dense_units
0.2               |0.4               |dropout
0.0028087         |0.00015601        |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
11/70 [===>..........................] - ETA: 2:07 - loss: 10.0684 - accuracy: 0.3949

KeyboardInterrupt: 

In [ ]:
best_hyperparameters_hyperband = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperband Model Hyperparameters:", best_hyperparameters_hyperband.values)
best_trial_hyperband = hyperband_tuner.oracle.get_best_trials(num_trials=1)[0]
print("Best Validation Accuracy Using Hyperband:", best_trial_hyperband.score)

Best Hyperband Model Hyperparameters: {'conv_1_filters': 64, 'conv_2_filters': 256, 'conv_3_filters': 416, 'dense_units': 512, 'dropout': 0.4, 'learning_rate': 0.00015600773861619983, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
Best Validation Accuracy Using Hyperband: 0.70705246925354


In [ ]:
# If you also want to evaluate and print training accuracy, you can do so by using:
train_loss, train_accuracy = best_model.evaluate(train_generator)
print("Training Accuracy Using Random Search:", train_accuracy)

# For the validation set:
validation_loss, validation_accuracy = best_model.evaluate(validation_generator)
print("Validation Accuracy Using Random Search:", validation_accuracy)